In [1]:
!pip install faiss-cpu

import faiss
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 27.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model = SentenceTransformer("all-mpnet-base-v2")

model = model.to(device)

catalog_df = pd.read_csv("/content/drive/MyDrive/Zepto IDC Query Classification/Zepto Data Challenge_ Intent prediction - catalog.csv")
catalog_df = catalog_df.drop_duplicates(subset=["product_variant_id"])

labeled_df = pd.read_csv("/content/drive/MyDrive/Zepto IDC Query Classification/Zepto Data Challenge_ Intent prediction - labelled_data.csv")

Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import pandas as pd
import ast
import random

def generate_contrastive_pairs_with_levels(df, num_negatives=2):
    all_l1, all_l2, all_l3 = set(), set(), set()

    # Convert stringified lists to Python lists
    df["l1_preds"] = df["l1_preds"].apply(ast.literal_eval)
    df["l2_preds"] = df["l2_preds"].apply(ast.literal_eval)
    df["l3_preds"] = df["l3_preds"].apply(ast.literal_eval)

    # Collect all category values
    for _, row in df.iterrows():
        all_l1.update(row["l1_preds"])
        all_l2.update(row["l2_preds"])
        all_l3.update(row["l3_preds"])

    all_l1, all_l2, all_l3 = list(all_l1), list(all_l2), list(all_l3)

    contrastive_data = []

    for _, row in df.iterrows():
        query = row["search_term"]

        # L1 positives
        for l1 in row["l1_preds"]:
            contrastive_data.append({"query": f"L1: {query}", "category": f"L1: {l1}", "label": 1.0})
        neg_l1s = random.sample([x for x in all_l1 if x not in row["l1_preds"]], min(num_negatives, len(all_l1)))
        for l1 in neg_l1s:
            contrastive_data.append({"query": f"L1: {query}", "category": f"L1: {l1}", "label": 0.0})

        # L2 positives
        for l2 in row["l2_preds"]:
            contrastive_data.append({"query": f"L2: {query}", "category": f"L2: {l2}", "label": 1.0})
        neg_l2s = random.sample([x for x in all_l2 if x not in row["l2_preds"]], min(num_negatives, len(all_l2)))
        for l2 in neg_l2s:
            contrastive_data.append({"query": f"L2: {query}", "category": f"L2: {l2}", "label": 0.0})

        # L3 positives
        for l3 in row["l3_preds"]:
            contrastive_data.append({"query": f"L3: {query}", "category": f"L3: {l3}", "label": 1.0})
        neg_l3s = random.sample([x for x in all_l3 if x not in row["l3_preds"]], min(num_negatives, len(all_l3)))
        for l3 in neg_l3s:
            contrastive_data.append({"query": f"L3: {query}", "category": f"L3: {l3}", "label": 0.0})

    return pd.DataFrame(contrastive_data)


In [5]:
contrastive_df = generate_contrastive_pairs_with_levels(labeled_df)
contrastive_df.to_csv("contrastive_train.csv", index=False)


In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [7]:
contrastive_df

,query,category,label
0,L1: atta maggi,L1: Packaged Food,1.0
1,L1: atta maggi,L1: Stationery & Books,0.0
2,L1: atta maggi,L1: Baby Care,0.0
3,L2: atta maggi,L2: Noodles & Vermicelli,1.0
4,L2: atta maggi,L2: Dessert Mixes,0.0
...,...,...,...
145487,L2: colored a4 sheets,L2: Paneer & Cream,0.0
145488,L3: colored a4 sheets,L3: Craft paper,1.0
145489,L3: colored a4 sheets,L3: Scribble Paper,1.0
145490,L3: colored a4 sheets,L3: Clutch,0.0


In [12]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import torch

# Load tokenizer
model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)


hf_dataset = Dataset.from_list(contrastive_df.to_dict(orient="records"))


In [13]:
# Tokenize
def tokenize_example(example):
    tokenized = tokenizer(
        example['query'],
        example['category'],
        truncation=True,
        padding='max_length',
        max_length=128
    )
    tokenized['labels'] = int(example['label'])
    return tokenized

In [14]:
hf_dataset = hf_dataset.map(tokenize_example, batched=False)

# Load base model
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Apply LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

model = get_peft_model(base_model, lora_config)

Map:   0%|          | 0/145492 [00:00<?, ? examples/s]

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:


# Training args
training_args = TrainingArguments(
    output_dir="./l123-lora-checkpoints",
    per_device_train_batch_size=32,
    learning_rate=2e-4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none"
)

# Custom trainer to use cross-entropy loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels").long()  # ✅ Fix here: ensure labels are int
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, 2), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Train
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset,
    tokenizer=tokenizer
)

trainer.train()

# Save final model
model.save_pretrained("/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned")
tokenizer.save_pretrained("/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned")


<ipython-input-17-023ad21180bc>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,0.556300
1000,0.464700


Step,Training Loss
500,0.556300
1000,0.464700
1500,0.439100
2000,0.416700
2500,0.414800
3000,0.400400
3500,0.393600
4000,0.390000
4500,0.391900
5000,0.372400


('/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned/tokenizer_config.json',
 '/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned/special_tokens_map.json',
 '/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned/vocab.txt',
 '/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned/added_tokens.json',
 '/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned/tokenizer.json')

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel

model_path = "/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned"

tokenizer = AutoTokenizer.from_pretrained(model_path)
base_model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-mpnet-base-v2", num_labels=2)
model = PeftModel.from_pretrained(base_model, model_path)
model.eval()


Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): MPNetForSequenceClassification(
      (mpnet): MPNetModel(
        (embeddings): MPNetEmbeddings(
          (word_embeddings): Embedding(30527, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): MPNetEncoder(
          (layer): ModuleList(
            (0-11): 12 x MPNetLayer(
              (attention): MPNetAttention(
                (attn): MPNetSelfAttention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_feature

In [ ]:
def generate_embeddings(text_list, tokenizer, model, batch_size=32, max_length=128):
    model.eval()
    embeddings = []

    with torch.no_grad():
        for i in range(0, len(text_list), batch_size):
            batch = text_list[i:i+batch_size]
            encoded = tokenizer(
                batch,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=max_length
            )
            input_ids = encoded["input_ids"].to(model.device)
            attention_mask = encoded["attention_mask"].to(model.device)

            outputs = model.base_model.base_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True,
                return_dict=True
            )

            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
            embeddings.append(cls_embeddings.cpu())

    return torch.cat(embeddings, dim=0)


In [9]:
# Define embedding function using CLS token
def generate_embeddings(text_list, tokenizer, model, batch_size=32, max_length=128):
    device = model.device
    embeddings = []

    with torch.no_grad():
        for i in tqdm(range(0, len(text_list), batch_size)):
            batch = text_list[i:i+batch_size]
            encoded = tokenizer(
                batch,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=max_length
            )
            input_ids = encoded["input_ids"].to(device)
            attention_mask = encoded["attention_mask"].to(device)

            outputs = model.base_model.base_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True,
                return_dict=True
            )

            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            embeddings.append(cls_embeddings.cpu())

    return torch.cat(embeddings, dim=0)

In [11]:
catalog_df

,product_variant_id,product_name,l1_name,l2_name,l3_name
0,071df257-1df3-4816-86fc-b52565aa87d2,himalaya baby lotion,Baby Care,Baby Skin & Hair Care,Baby Lotion
1,ebc89978-cd10-413a-bb76-06ff88b64788,himalaya baby lotion,Baby Care,Baby Skin & Hair Care,Baby Lotion
2,2c157578-52ac-4ac6-84c0-d24747d8c894,johnson's baby lotion,Baby Care,Baby Skin & Hair Care,Baby Lotion
3,98417151-e799-4aa4-a49e-e211609662df,cetaphil baby daily lotion,Baby Care,Baby Skin & Hair Care,Baby Lotion
4,e7fc1148-8811-477f-8cd6-d5891ec70b4f,"sebamed baby body lotion, for all skin types",Baby Care,Baby Skin & Hair Care,Baby Lotion
...,...,...,...,...,...
8887,e3b84007-a4a9-45d5-9fbd-9e6a06cf5544,watermelon kiran (dharboosani),Fruits & Vegetables,Fresh Fruits,Watermelon
8888,c1faa6cf-42a3-4a88-be2c-deda79892308,watermelon regular - dharboosani,Fruits & Vegetables,Fresh Fruits,Watermelon
8889,af6fa6e7-ff15-4a2b-afc7-6a791bb013a0,watermelon kiran (dharboosani),Fruits & Vegetables,Fresh Fruits,Watermelon
8890,767bc7e5-5806-43b2-8e55-5aeccecb16d9,watermelon kiran,Fruits & Vegetables,Fresh Fruits,Watermelon


In [12]:
labeled_df

,search_term,l1_preds,l2_preds,l3_preds
0,atta maggi,[Packaged Food],[Noodles & Vermicelli],"[Instant Noodles, Bundle]"
1,bombay shaving company,"[Electronics & Appliances, Feminine Hygiene, F...","[Grooming, Hair Removal For Women, Shaving Nee...","[Trimmer, Razor, Shaving Cream, Face Wash, Sha..."
2,choco fills,"[Biscuits, Sweet Cravings, Breakfast & Sauces,...","[Chocolates, Breakfast Cereals, Creamfills, Pa...","[Wafer Chocolates, Kids' Cereal, Centre Fills,..."
3,chocos,"[Biscuits, Sweet Cravings, Breakfast & Sauces]","[Cookies, Pastries & Cakes, Breakfast Cereals,...","[Choco Pie, Kids' Cereal]"
4,epigamia yogurt,"[Cold Drinks & Juices, Dairy, Bread & Eggs]","[Milk Drinks, Yogurts & Dahi]","[Smoothie, Dahi, Yogurts]"
...,...,...,...,...
10830,colgate small toothpaste,[Bath & Body],[Toothpaste & Mouthwash],[Toothpaste]
10831,colgate whitening toothpaste,[Bath & Body],[Toothpaste & Mouthwash],[Toothpaste]
10832,cologne mask,"[Fragrances & Grooming, Skincare, Apparel & Li...","[Perfumes, Face Care, Travel Accessories, Eye ...","[Body Perfume, Neck Pillow, Sheet Mask, Under ..."
10833,color fx,"[Makeup & Beauty, Fragrances & Grooming]","[Face Makeup, Hygiene Essentials, Nails, Gifti...","[Nail Polish Remover, Makeup Remover, Nail Pol..."


In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

# -----------------------------
# LOAD & PREPARE DATA
# -----------------------------
catalog_df = pd.read_csv("/content/drive/MyDrive/Zepto IDC Query Classification/Zepto Data Challenge_ Intent prediction - catalog.csv")
catalog_df = catalog_df.drop_duplicates(subset=["product_variant_id"])
labeled_df = pd.read_csv("/content/drive/MyDrive/Zepto IDC Query Classification/Zepto Data Challenge_ Intent prediction - labelled_data.csv")

# Format L1 inputs
catalog_df["l1_input"] = catalog_df.apply(lambda row: f"L1: {row['product_name']} | {row['l1_name']}", axis=1)
labeled_df["l1_query"] = labeled_df["search_term"].apply(lambda x: f"L1: {x}")

# -----------------------------
# LOAD MODEL & TOKENIZER
# -----------------------------
model_path = "/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)

base_model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-mpnet-base-v2", num_labels=2)
model = PeftModel.from_pretrained(base_model, model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

# -----------------------------
# EMBEDDING FUNCTION
# -----------------------------
def generate_embeddings(text_list, tokenizer, model, batch_size=32, max_length=128):
    embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(text_list), batch_size)):
            batch = text_list[i:i+batch_size]
            encoded = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
            input_ids = encoded["input_ids"].to(device)
            attention_mask = encoded["attention_mask"].to(device)

            outputs = model.base_model.base_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True,
                return_dict=True
            )

            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
            embeddings.append(cls_embeddings.cpu())
    return torch.cat(embeddings, dim=0)

# -----------------------------
# GENERATE EMBEDDINGS
# -----------------------------
catalog_l1_inputs = catalog_df["l1_input"].tolist()
query_l1_inputs = labeled_df["l1_query"].tolist()

catalog_l1_embeddings = generate_embeddings(catalog_l1_inputs, tokenizer, model)
query_l1_embeddings = generate_embeddings(query_l1_inputs, tokenizer, model)

# Optional: Save to disk or keep for FAISS
# torch.save(catalog_l1_embeddings, "catalog_l1_embeddings.pt")
# torch.save(query_l1_embeddings, "query_l1_embeddings.pt")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 339/339 [00:07<00:00, 44.37it/s]


In [21]:
import ast
import re
def parse_list_string(text):
    if pd.isna(text) or text == '':
        return []
    try:
        # Handle cases where the list is represented as a string
        return ast.literal_eval(text)
    except (ValueError, SyntaxError):
        # Try to extract strings between quotes using regex
        matches = re.findall(r'"([^"]*)"', text)
        if matches:
            return matches
        return []

In [24]:
# -----------------------------
# PROCESS L1 CATEGORIES
# -----------------------------
print("Processing L1 categories...")

# Extract all unique L1 categories from catalog
catalog_l1_df = catalog_df[['product_variant_id', 'product_name', 'l1_name']].copy()
catalog_l1_df.rename(columns={'l1_name': 'category'}, inplace=True)
catalog_l1_df['source'] = 'catalog'
catalog_l1_df['level'] = 'L1'

# Extract all L1 categories from labeled data
labeled_l1_rows = []
for idx, row in labeled_df.iterrows():
    search_term = row['search_term']
    for l1 in row['l1_preds']:
        labeled_l1_rows.append({
            'product_name': search_term,
            'category': l1,
            'source': 'labeled',
            'level': 'L1'
        })

labeled_l1_df = pd.DataFrame(labeled_l1_rows)

# Combine catalog and labeled L1 data
all_l1_df = pd.concat([
    catalog_l1_df[['product_name', 'category', 'source', 'level']],
    labeled_l1_df[['product_name', 'category', 'source', 'level']]
])

# Generate L1 embeddings
# Final format: "passage: L1 product_name | L1 category"
def format_l1_text(row):
    return f"passage: L1 {row['product_name']} | L1 {row['category']}"

all_l1_texts = all_l1_df.apply(format_l1_text, axis=1).tolist()



Processing L1 categories...


In [26]:
# -----------------------------
# PROCESS L2 CATEGORIES
# -----------------------------
print("Processing L2 categories...")

# Extract all unique L2 categories from catalog
catalog_l2_df = catalog_df[['product_variant_id', 'product_name', 'l2_name']].copy()
catalog_l2_df.rename(columns={'l2_name': 'category'}, inplace=True)
catalog_l2_df['source'] = 'catalog'
catalog_l2_df['level'] = 'L2'

# Extract all L2 categories from labeled data
labeled_l2_rows = []
for idx, row in labeled_df.iterrows():
    search_term = row['search_term']
    for l2 in row['l2_preds']:
        labeled_l2_rows.append({
            'product_name': search_term,
            'category': l2,
            'source': 'labeled',
            'level': 'L2'
        })

labeled_l2_df = pd.DataFrame(labeled_l2_rows)

# Combine catalog and labeled L2 data
all_l2_df = pd.concat([
    catalog_l2_df[['product_name', 'category', 'source', 'level']],
    labeled_l2_df[['product_name', 'category', 'source', 'level']]
])

# Generate L2 embeddings
def format_l2_text(row):
    return f"passage: L2 {row['product_name']} | L2 {row['category']}"

all_l2_texts = all_l2_df.apply(format_l2_text, axis=1).tolist()

Processing L2 categories...


In [27]:
# -----------------------------
# PROCESS L3 CATEGORIES
# -----------------------------
print("Processing L3 categories...")

# Extract all unique L3 categories from catalog
catalog_l3_df = catalog_df[['product_variant_id', 'product_name', 'l3_name']].copy()
catalog_l3_df.rename(columns={'l3_name': 'category'}, inplace=True)
catalog_l3_df['source'] = 'catalog'
catalog_l3_df['level'] = 'L3'

# Extract all L3 categories from labeled data
labeled_l3_rows = []
for idx, row in labeled_df.iterrows():
    search_term = row['search_term']
    for l3 in row['l3_preds']:
        labeled_l3_rows.append({
            'product_name': search_term,
            'category': l3,
            'source': 'labeled',
            'level': 'L3'
        })

labeled_l3_df = pd.DataFrame(labeled_l3_rows)

# Combine catalog and labeled L3 data
all_l3_df = pd.concat([
    catalog_l3_df[['product_name', 'category', 'source', 'level']],
    labeled_l3_df[['product_name', 'category', 'source', 'level']]
])

# Generate L3 embeddings
def format_l3_text(row):
    return f"passage: L3{row['product_name']} | L3{row['category']}"

all_l3_texts = all_l3_df.apply(format_l3_text, axis=1).tolist()


Processing L3 categories...


In [29]:
model_path = "/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)

base_model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-mpnet-base-v2", num_labels=2)
model = PeftModel.from_pretrained(base_model, model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
def generate_embeddings(text_list, tokenizer, model, batch_size=32, max_length=128):
    embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(text_list), batch_size)):
            batch = text_list[i:i+batch_size]
            encoded = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
            input_ids = encoded["input_ids"].to(device)
            attention_mask = encoded["attention_mask"].to(device)

            outputs = model.base_model.base_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True,
                return_dict=True
            )

            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
            embeddings.append(cls_embeddings.cpu())
    return torch.cat(embeddings, dim=0)


In [31]:
# Assuming `format_l1_text`, `format_l2_text`, and `format_l3_text` are defined
# Generate the text lists (if not already done)
all_l1_texts = all_l1_df.apply(format_l1_text, axis=1).tolist()
all_l2_texts = all_l2_df.apply(format_l2_text, axis=1).tolist()
all_l3_texts = all_l3_df.apply(format_l3_text, axis=1).tolist()

# Generate embeddings for L1, L2, and L3 texts
l1_embeddings = generate_embeddings(all_l1_texts, tokenizer, model)
l2_embeddings = generate_embeddings(all_l2_texts, tokenizer, model)
l3_embeddings = generate_embeddings(all_l3_texts, tokenizer, model)

# Move embeddings to CPU after computation (if needed) and convert to NumPy arrays
l1_embeddings = l1_embeddings.cpu().numpy()
l2_embeddings = l2_embeddings.cpu().numpy()
l3_embeddings = l3_embeddings.cpu().numpy()

# Save embeddings if needed (for example, in a pandas DataFrame)
l1_df = pd.DataFrame(l1_embeddings)
l2_df = pd.DataFrame(l2_embeddings)
l3_df = pd.DataFrame(l3_embeddings)

# Save to CSV for later use
l1_df.to_csv("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/l1_embeddings.csv", index=False)
l2_df.to_csv("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/l2_embeddings.csv", index=False)
l3_df.to_csv("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/l3_embeddings.csv", index=False)

# If you need to perform further analysis, you can now load the CSVs


  2%|▏         | 18/1067 [01:05<1:04:04,  3.66s/it]


KeyboardInterrupt: 

In [ ]:
import numpy as np
np.save("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/all_l1_embeddings.npy", l1_embeddings)
np.save("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/all_l2_embeddings.npy", l2_embeddings)
np.save("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/all_l3_embeddings.npy", l3_embeddings)

print(f"Saved embeddings: L1={len(l1_embeddings)}, L2={len(l2_embeddings)}, L3={len(l3_embeddings)}")
print("Complete!")

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 33.8 MB/s eta 0:00:00


In [3]:
import faiss
import numpy as np

# Load your L1 embeddings (from .npy or memory)
l1_embeddings = np.load("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/all_l1_embeddings.npy")

# Normalize (FAISS works better with normalized vectors + cosine similarity)
faiss.normalize_L2(l1_embeddings)

# Build FAISS index
index = faiss.IndexFlatIP(l1_embeddings.shape[1])  # IP = inner product (cosine if normalized)
index.add(l1_embeddings)

# Save the index
faiss.write_index(index, "/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/faiss_l1.index")

print(f"FAISS index built and saved. Total entries: {index.ntotal}")


FAISS index built and saved. Total entries: 31207


In [7]:
def embed_query(text, tokenizer, model):
    model.eval()
    with torch.no_grad():
        encoded = tokenizer(
            [text],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        )
        input_ids = encoded["input_ids"].to(model.device)
        attention_mask = encoded["attention_mask"].to(model.device)

        outputs = model.base_model.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True
        )

        cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
        return cls_embedding.cpu().numpy()

In [8]:
index = faiss.read_index("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/faiss_l1.index")


In [9]:
def search_l1(query, all_l1_df, tokenizer, model, index, k=10):
    input_text = f"L1 {query}"
    embedding = embed_query(input_text, tokenizer, model)
    faiss.normalize_L2(embedding)

    scores, indices = index.search(embedding, k)
    results = []
    for score, idx in zip(scores[0], indices[0]):
        category = all_l1_df.iloc[idx]["category"]
        product = all_l1_df.iloc[idx]["product_name"]
        results.append((round(score, 4), category, product))
    return results

In [10]:
query = "atta maggi"
top_results = search_l1(query, all_l1_df, tokenizer, model, index)

print(f"\n🔍 Top L1 categories for: '{query}'\n")
for score, category, product in top_results:
    print(f"[{score}] {category} (product: {product})")


NameError: name 'all_l1_df' is not defined